In [1]:
# 경고창 무시
import warnings
warnings.filterwarnings('ignore')

## 피클 불러오기

In [2]:
jh_pickle = 'svc_best_model_jh.pkl'
sc_pickle = 'svc_best_model_sc.pkl'
mj_pickle = 'svc_best_model_mj.pkl'

import pickle
with open(jh_pickle,'rb') as pickle_file:
  jh_classification_model = pickle.load(pickle_file)
with open(sc_pickle,'rb') as pickle_file:
  sc_classification_model = pickle.load(pickle_file)
with open(mj_pickle,'rb') as pickle_file:
  mj_classification_model = pickle.load(pickle_file)

## 데이터 불러오기

In [3]:
import pandas as pd
import pymongo
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import scipy
import scipy.stats as stats
from konlpy.tag import Okt
okt = Okt()
from mecab import MeCab
mecab = MeCab()
import pyLDAvis
import pyLDAvis.lda_model
     
client=pymongo.MongoClient("192.168.10.240",27017)
db= client['AI_LKJ']
collection_tour = db['tour_contents']
collection_tour_train = db['data_tour_NLP_train']
collection_attraction = db['attraction_place_2023']

data_tour = collection_tour.find()
data_tour_train = collection_tour_train.find()
data_attraction = collection_attraction.find()

df_tour = pd.DataFrame(data_tour)
df_tour_train = pd.DataFrame(data_tour_train)
df_attraction = pd.DataFrame(data_attraction)
df_tour

,_id,review_text,지역
0,65ee5e2b901db79521bc8779,[이벤트] 동트는 강원 취향저격 이벤트! 나에게 딱 맞는 강원 네이처 로드 코스는?,강원
1,65ee5e2b901db79521bc877a,봄 오는 소리 귀 기울이고 겨울 가는 아쉬움 녹이고,강원
2,65ee5e2c901db79521bc877b,하얀 이불 벗고 봄으로 가는 길 태평성대 가져온 봉황 눈앞에,강원
3,65ee5e2c901db79521bc877c,"열흘에 한 번, 고원 마을에 열리는 반짝 세일! 태백 통리장",강원
4,65ee5e2c901db79521bc877d,걸음 걸음 쌓이는 설렘 바다로 가는 오름길,강원
...,...,...,...
49657,65ee9fa9c27eda916fc759b4,집에서 만나는 전국 맛집,제주
49658,65ee9faac27eda916fc759b5,[윈터 바캉스] 10만원으로 떠나는 제주 당일치기,제주
49659,65ee9faac27eda916fc759b6,제주 서귀포층 패류 화석산지,제주
49660,65ee9faac27eda916fc759b7,제주 한림 용암동굴지대,제주


In [4]:
def split(text):
    clean_text = []
    for word in text.split():
        if "기자" not in word:
            clean_text.append(word)
    return " ".join(clean_text)

In [5]:
df_tour.loc[:,'review_text'] = df_tour['review_text'].apply(split)
df_tour.query("review_text.str.contains('자단')")

,_id,review_text,지역


In [6]:
# 토크나이징 함수 정의
from konlpy.tag import Okt
okt = Okt()
stopwords = ['남해','대전','대전','공주','안동','나주','바퀴','보령','고령','신안','특산','공항','곡성','신상','사람','위치','가지','진도','장동','정리','도미','고창','모음','장성','시작','남원','가격','화순','국립','베스트','오토','레이스','후기','피아','중앙','공모전','태안','성주','봉화','남도','목포','군산','구례','순천','문경','완주','청주','청도','여수','울주군','포구','두기','거리','강진','뉴스','방법','통영','부여','한밭수목원','거제','서쪽','동쪽','양산','하동','포항','진주','우도','서귀포','당신', '대관령','속초','특별자치도', '삼척','영월','원주','화천','고성','양양','춘천','양구','철원','홍천','태백','달성군','구석구석','블로그','여행','여행지','경기','여주','서구','현황', '장생포','양평','가창','남평문씨','파주','영상','소개','지역','명소','요즘','마을','달성','한눈','카드','추천','수성','안산','기회','횡성','문진','발생','코스','현풍','문진','만끽']
def Tokenizer(raw, pos=["Noun"], stopword=stopwords):
    word_list = []
    for word, tag in okt.pos(raw,norm=True,stem=True):
        if len(word) > 1 and tag in pos and word not in stopword:
            if mecab.pos(word)[0][1] in ["NNG"]:
                if word == '화관':
                    word = '문화관'
                word_list.append(word)
    return " ".join(word_list)

In [7]:
df_tour.loc[:,'review_text_clean']=df_tour['review_text'].apply(Tokenizer)

In [8]:
df_tour

,_id,review_text,지역,review_text_clean
0,65ee5e2b901db79521bc8779,[이벤트] 동트는 강원 취향저격 이벤트! 나에게 딱 맞는 강원 네이처 로드 코스는?,강원,이벤트 취향 저격 이벤트 로드
1,65ee5e2b901db79521bc877a,봄 오는 소리 귀 기울이고 겨울 가는 아쉬움 녹이고,강원,소리 겨울 아쉬움
2,65ee5e2c901db79521bc877b,하얀 이불 벗고 봄으로 가는 길 태평성대 가져온 봉황 눈앞에,강원,이불 태평성대 봉황 눈앞
3,65ee5e2c901db79521bc877c,"열흘에 한 번, 고원 마을에 열리는 반짝 세일! 태백 통리장",강원,열흘 통리
4,65ee5e2c901db79521bc877d,걸음 걸음 쌓이는 설렘 바다로 가는 오름길,강원,걸음 걸음 바다로
...,...,...,...,...
49657,65ee9fa9c27eda916fc759b4,집에서 만나는 전국 맛집,제주,전국 맛집
49658,65ee9faac27eda916fc759b5,[윈터 바캉스] 10만원으로 떠나는 제주 당일치기,제주,바캉스 당일치기
49659,65ee9faac27eda916fc759b6,제주 서귀포층 패류 화석산지,제주,패류 화석 산지
49660,65ee9faac27eda916fc759b7,제주 한림 용암동굴지대,제주,한림 용암동굴 지대


## 데이터 분류하기

In [9]:
df_tour_train

,_id,concept,season,review_text,지역,review_text_clean
0,65ee6316a64dee66ed301e2a,0,0,[블로그 울산 남구 아이스팩 전용 수거함 어디에 있을까?,울산,아이스팩 전용 수거
1,65ee670ba64dee66ed303175,5,3,[가을여행] 전주 당일치기 여행 BEST10 (feat. 덕진공원팸투어) /가을 단...,전북,가을 전주 당일치기 진공 투어 가을 단풍
2,65ee6a64c528b55b0ae79e58,4,0,"대전 유성구 관평동 맛집, 감동적인 맛과 가격 '매봉 쌀국수와 소바'",대전,맛집 감동 쌀국수
3,65ee60fc901db79521bc991b,3,0,역사를 따라가는 DMZ 로드,경기,역사 로드
4,65ee664fc528b55b0ae78d7a,5,0,"아브라다카브라라는 주문을 외워보고 싶은, '대전 보라매공원크리스마스 풍경'",대전,주문 보라매 공원 풍경
...,...,...,...,...,...,...
9995,65ee9b78c27eda916fc74ead,5,-,서귀포 추천 여행지 신창풍차해안도로,제주,풍차 해안 도로
9996,65ee66b6901db79521bcb1d6,4,-,"반려견과 함께 하는 커피 한 잔, 광주 애견동반카페 모음.ZIP",광주,반려견 커피 애견 동반
9997,65ee6722a64dee66ed3032ef,1,-,전북 문화공간 - 완주 복합문화지구 누에(Nu-e),전북,문화 공간 복합 문화 지구 누에
9998,65ee6cd5c528b55b0ae7a7b4,5,-,"벚꽃은 갔지만 유채꽃 만발, 서울에서 만나는 남해",서울,벚꽃 유채꽃 만발


In [10]:
df_tour_train.isnull().sum()

_id                    0
concept                0
season                 0
review_text            0
지역                     0
review_text_clean    225
dtype: int64

In [11]:
df_tour_train=df_tour_train.dropna()

In [12]:
df_tour_train.isnull().sum()

_id                  0
concept              0
season               0
review_text          0
지역                   0
review_text_clean    0
dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
df_tour_train_train,df_tour_train_test = train_test_split(df_tour_train,test_size=0.3,random_state=42)
df_tour_train['concept'].unique()

array([0, 5, 4, 3, 1, 2, 6])

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfVectorizer = TfidfVectorizer(tokenizer = Tokenizer, use_idf=True,max_df=0.95,min_df=2,ngram_range=(1,2))
tfidfVectorizer.fit(df_tour_train['review_text_clean'])

TfidfVectorizer(max_df=0.95, min_df=2, ngram_range=(1, 2),
                tokenizer=<function Tokenizer at 0x7fb8649ee840>)

In [15]:
train_feature = tfidfVectorizer.transform(df_tour_train_train['review_text_clean']) # train
train_target = df_tour_train_train['concept'] # test
test_feature = tfidfVectorizer.transform(df_tour_train_test['review_text_clean']) # train
test_target = df_tour_train_test['concept']


In [16]:
df_tour_train

,_id,concept,season,review_text,지역,review_text_clean
0,65ee6316a64dee66ed301e2a,0,0,[블로그 울산 남구 아이스팩 전용 수거함 어디에 있을까?,울산,아이스팩 전용 수거
1,65ee670ba64dee66ed303175,5,3,[가을여행] 전주 당일치기 여행 BEST10 (feat. 덕진공원팸투어) /가을 단...,전북,가을 전주 당일치기 진공 투어 가을 단풍
2,65ee6a64c528b55b0ae79e58,4,0,"대전 유성구 관평동 맛집, 감동적인 맛과 가격 '매봉 쌀국수와 소바'",대전,맛집 감동 쌀국수
3,65ee60fc901db79521bc991b,3,0,역사를 따라가는 DMZ 로드,경기,역사 로드
4,65ee664fc528b55b0ae78d7a,5,0,"아브라다카브라라는 주문을 외워보고 싶은, '대전 보라매공원크리스마스 풍경'",대전,주문 보라매 공원 풍경
...,...,...,...,...,...,...
9995,65ee9b78c27eda916fc74ead,5,-,서귀포 추천 여행지 신창풍차해안도로,제주,풍차 해안 도로
9996,65ee66b6901db79521bcb1d6,4,-,"반려견과 함께 하는 커피 한 잔, 광주 애견동반카페 모음.ZIP",광주,반려견 커피 애견 동반
9997,65ee6722a64dee66ed3032ef,1,-,전북 문화공간 - 완주 복합문화지구 누에(Nu-e),전북,문화 공간 복합 문화 지구 누에
9998,65ee6cd5c528b55b0ae7a7b4,5,-,"벚꽃은 갔지만 유채꽃 만발, 서울에서 만나는 남해",서울,벚꽃 유채꽃 만발


## Season 모델 작성
- season_RFC

In [17]:
df_tour_train.query("season != '-'")['season'].unique()

array([0, 3, 4, 1, 2], dtype=object)

In [18]:
df_tour_train_train.query("season != '-'")['review_text_clean']

655           캠핑 구산 해수욕장
178         엄마 학교 신입생 모집
3205      봉유 돼지 짬뽕 전복 짬뽕
800             기념물 이팝나무
92                항공샷 랜선
              ...       
2481         마다 평온 고이 시간
782      벚꽃 폴리스 공원 봄꽃 산책
1723         대비 면역 제철 음식
477     혈관 온라인 건강 캠페인 음식
878       외로움 간직 박물관 비양도
Name: review_text_clean, Length: 2727, dtype: object

In [19]:
season_train_feature = tfidfVectorizer.transform(df_tour_train_train.query("season != '-'")['review_text_clean']) # train
season_train_target = df_tour_train_train.query("season != '-'")['season'].astype('str') # test
season_train_feature_test = tfidfVectorizer.transform(df_tour_train_test.query("season != '-'")['review_text_clean']) # train
season_train_target_test = df_tour_train_test.query("season != '-'")['season'].astype('str') # test

In [20]:
from sklearn.ensemble import RandomForestClassifier
season_RFC=RandomForestClassifier(n_jobs=-1)
season_RFC.fit(season_train_feature,season_train_target)

RandomForestClassifier(n_jobs=-1)

In [21]:
from sklearn.metrics import accuracy_score
season_train_feature_predict = season_RFC.predict(season_train_feature)
accuracy_score(season_train_target, season_train_feature_predict)

0.9996332966629996

In [22]:
from sklearn.metrics import classification_report
print(classification_report(season_train_target, season_train_feature_predict))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2273
           1       1.00      1.00      1.00       104
           2       0.99      1.00      1.00       103
           3       1.00      1.00      1.00       148
           4       1.00      1.00      1.00        99

    accuracy                           1.00      2727
   macro avg       1.00      1.00      1.00      2727
weighted avg       1.00      1.00      1.00      2727



In [23]:
season_test_feature_predict = season_RFC.predict(season_train_feature_test)
accuracy_score(season_train_target_test, season_test_feature_predict)

0.966301600673968

In [24]:
print(classification_report(season_train_target_test, season_test_feature_predict))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       966
           1       1.00      0.76      0.86        41
           2       0.98      0.88      0.93        52
           3       0.95      0.89      0.92        81
           4       0.93      0.85      0.89        47

    accuracy                           0.97      1187
   macro avg       0.96      0.87      0.92      1187
weighted avg       0.97      0.97      0.97      1187



## Concept 모델 선택하기

### 주현

In [25]:
jh_train_feature = tfidfVectorizer.transform(df_tour_train_train['review_text_clean']) # train
jh_train_target = df_tour_train_train['concept'] # test
jh_test_feature = tfidfVectorizer.transform(df_tour_train_test['review_text_clean']) # train
jh_test_target = df_tour_train_test['concept']

In [26]:
jh_classification_model.fit(jh_train_feature,jh_train_target)

SVC(C=1, degree=2, gamma=0.1, kernel='linear')

In [27]:
jh_train_feature_predict = jh_classification_model.predict(jh_train_feature)

In [28]:
from sklearn.metrics import classification_report
print(classification_report(jh_train_target,jh_train_feature_predict))

              precision    recall  f1-score   support

           0       0.82      0.93      0.87      1945
           1       0.90      0.84      0.87      1656
           2       0.95      0.78      0.86       195
           3       0.89      0.76      0.82       325
           4       0.96      0.89      0.92       593
           5       0.89      0.90      0.89      1832
           6       0.84      0.71      0.77       296

    accuracy                           0.87      6842
   macro avg       0.89      0.83      0.86      6842
weighted avg       0.88      0.87      0.87      6842



In [29]:
jh_test_feature_predict = jh_classification_model.predict(jh_test_feature)

In [30]:
from sklearn.metrics import classification_report
print(classification_report(jh_test_target,jh_test_feature_predict))

              precision    recall  f1-score   support

           0       0.71      0.85      0.77       833
           1       0.80      0.72      0.76       729
           2       0.87      0.55      0.67        73
           3       0.79      0.55      0.65       138
           4       0.89      0.75      0.81       232
           5       0.78      0.83      0.80       793
           6       0.70      0.53      0.60       135

    accuracy                           0.77      2933
   macro avg       0.79      0.68      0.72      2933
weighted avg       0.77      0.77      0.76      2933



### 명준

In [31]:
# 학습시킬 데이터
mj_train_feature = tfidfVectorizer.transform(df_tour_train_train['review_text_clean']) # train
mj_train_target = df_tour_train_train['concept'] # train
# 테스트 할 데이터
mj_test_feature = tfidfVectorizer.transform(df_tour_train_test['review_text_clean']) # test
mj_test_target = df_tour_train_test['concept'] # test
# review_text_clean를 통해 concept을 예측. 즉 feature의 데이터를 통해 target을 예측한다.

In [32]:
# mj_classification_model 모델로 mj_train_feature와 mj_train_target 데이터를 사용해서 학습시킨다.
mj_classification_model.fit(mj_train_feature,mj_train_target)

GradientBoostingClassifier(n_estimators=5000)

In [33]:
# 학습된 결과가 얼마나 잘 적용되었는지 평가한다.
mj_train_feature_predict = mj_classification_model.predict(mj_train_feature)

In [34]:
# 출력 결과를 확인하기(정밀도, 재현율,F1, 정확도, 클래스 별 결과)
from sklearn.metrics import classification_report
print(classification_report(mj_train_target,mj_train_feature_predict))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1945
           1       1.00      0.99      1.00      1656
           2       0.99      1.00      1.00       195
           3       0.99      0.99      0.99       325
           4       1.00      1.00      1.00       593
           5       1.00      0.99      1.00      1832
           6       0.98      1.00      0.99       296

    accuracy                           0.99      6842
   macro avg       0.99      1.00      0.99      6842
weighted avg       0.99      0.99      0.99      6842



In [35]:
# 학습된 결과가 얼마나 잘 적용되었는지 평가한다.
mj_test_feature_predict = mj_classification_model.predict(mj_test_feature)

In [36]:
# 출력 결과를 확인하기(정밀도, 재현율,F1, 정확도, 클래스 별 결과)
from sklearn.metrics import classification_report
print(classification_report(mj_test_target,mj_test_feature_predict))

              precision    recall  f1-score   support

           0       0.71      0.81      0.76       833
           1       0.78      0.72      0.75       729
           2       0.61      0.64      0.63        73
           3       0.74      0.61      0.67       138
           4       0.86      0.77      0.81       232
           5       0.78      0.79      0.79       793
           6       0.55      0.47      0.50       135

    accuracy                           0.75      2933
   macro avg       0.72      0.69      0.70      2933
weighted avg       0.75      0.75      0.75      2933



### 석천

In [37]:
sc_train_feature = tfidfVectorizer.transform(df_tour_train_train['review_text_clean']) # train
sc_train_target = df_tour_train_train['concept'] # test
sc_test_feature = tfidfVectorizer.transform(df_tour_train_test['review_text_clean']) # train
sc_test_target = df_tour_train_test['concept']

In [38]:
sc_classification_model.fit(sc_train_feature,sc_train_target)

SVC(C=0.1, gamma=0.1, kernel='sigmoid')

In [39]:
sc_train_feature_predict = sc_classification_model.predict(sc_train_feature)

In [40]:
from sklearn.metrics import classification_report
print(classification_report(sc_train_target,sc_train_feature_predict))

              precision    recall  f1-score   support

           0       0.29      1.00      0.45      1945
           1       0.67      0.00      0.00      1656
           2       0.00      0.00      0.00       195
           3       0.00      0.00      0.00       325
           4       0.00      0.00      0.00       593
           5       0.90      0.03      0.06      1832
           6       0.00      0.00      0.00       296

    accuracy                           0.29      6842
   macro avg       0.27      0.15      0.07      6842
weighted avg       0.49      0.29      0.14      6842



In [41]:
sc_test_feature_predict = sc_classification_model.predict(sc_test_feature)

In [42]:
from sklearn.metrics import classification_report
print(classification_report(sc_test_target,sc_test_feature_predict))

              precision    recall  f1-score   support

           0       0.29      1.00      0.45       833
           1       1.00      0.00      0.00       729
           2       0.00      0.00      0.00        73
           3       0.00      0.00      0.00       138
           4       0.00      0.00      0.00       232
           5       0.78      0.03      0.05       793
           6       0.00      0.00      0.00       135

    accuracy                           0.29      2933
   macro avg       0.29      0.15      0.07      2933
weighted avg       0.54      0.29      0.14      2933



## 데이터 분석

In [48]:
## 컨셉(concept)
# 벡터라이징
tour_concept_feature=tfidfVectorizer.transform(df_tour['review_text_clean'])
# 컨셉 예측 모델 학습
tour_concept_predict = jh_classification_model.predict(tour_concept_feature)
tour_concept_season_RFC_predict = season_RFC.predict(tour_concept_feature)
# concept열 추가
df_tour.loc[:,'concept']=tour_concept_predict
# concept열 확인
df_tour

,_id,review_text,지역,review_text_clean,concept,season
0,65ee5e2b901db79521bc8779,[이벤트] 동트는 강원 취향저격 이벤트! 나에게 딱 맞는 강원 네이처 로드 코스는?,강원,이벤트 취향 저격 이벤트 로드,0,0
1,65ee5e2b901db79521bc877a,봄 오는 소리 귀 기울이고 겨울 가는 아쉬움 녹이고,강원,소리 겨울 아쉬움,0,4
2,65ee5e2c901db79521bc877b,하얀 이불 벗고 봄으로 가는 길 태평성대 가져온 봉황 눈앞에,강원,이불 태평성대 봉황 눈앞,0,0
3,65ee5e2c901db79521bc877c,"열흘에 한 번, 고원 마을에 열리는 반짝 세일! 태백 통리장",강원,열흘 통리,0,0
4,65ee5e2c901db79521bc877d,걸음 걸음 쌓이는 설렘 바다로 가는 오름길,강원,걸음 걸음 바다로,0,0
...,...,...,...,...,...,...
49657,65ee9fa9c27eda916fc759b4,집에서 만나는 전국 맛집,제주,전국 맛집,4,0
49658,65ee9faac27eda916fc759b5,[윈터 바캉스] 10만원으로 떠나는 제주 당일치기,제주,바캉스 당일치기,0,0
49659,65ee9faac27eda916fc759b6,제주 서귀포층 패류 화석산지,제주,패류 화석 산지,0,0
49660,65ee9faac27eda916fc759b7,제주 한림 용암동굴지대,제주,한림 용암동굴 지대,5,0


In [59]:
tour_concept_feature

<49662x6104 sparse matrix of type '<class 'numpy.float64'>'
	with 967037 stored elements in Compressed Sparse Row format>

In [58]:
## 계절(season)
# 벡터라이징
tour_concept_feature=tfidfVectorizer.transform(df_tour['review_text_clean'])
# 컨셉 예측 모델 학습
tour_concept_season_RFC_predict = season_RFC.predict(tour_concept_feature)
# season열 추가
df_tour.loc[:,'season']=tour_concept_season_RFC_predict
# season열 확인
df_tour

AttributeError: 'str' object has no attribute 'dtype'

## 데이터 탐색

In [50]:
df_tour.isnull().sum()

_id                  0
review_text          0
지역                   0
review_text_clean    0
concept              0
season               0
dtype: int64

In [51]:
# condition collection
concept_nothing = "concept == 0"
concept_culture = "concept == 1"
concept_leisure = "concept == 2"
concept_history = "concept == 3"
concept_food = "concept == 4"
concept_nature = "concept == 5"
concept_healing = "concept == 6"

season_nothing = "season == 0"
season_spring = "season == 1"
season_summer = "season == 2"
season_autumn = "season == 3"
season_winter = "season == 4"

In [52]:
# concept열에 0을 제외
df_tour_concept = df_tour[df_tour["concept"] != 0]

In [53]:
df_tour["season"]=df_tour["season"].astype('int')

In [54]:
df_tour_season = df_tour[df_tour["season"] != 0]

In [55]:
df_tour_concept

,_id,review_text,지역,review_text_clean,concept,season
7,65ee5e2c901db79521bc8780,"와인에 스민 청정자연, 향에 반하고 풍광에 취한다",강원,와인 청정 자연 취한,5,0
9,65ee5e2c901db79521bc8782,하늘에 뿌린 눈꽃 천상의 설원 되다,강원,하늘 눈꽃 천상 설원,5,0
11,65ee5e2d901db79521bc8784,"자연과 인공의 조화, 절벽 위에 핀 누정",강원,자연 인공 조화 절벽 누정,5,0
21,65ee5e2e901db79521bc878e,겨울 강원도여행 대관령 삼양라운드힐 설경 눈구경 가볼만한곳 삼양목장,강원,겨울 설경 구경 목장,5,4
23,65ee5e2e901db79521bc8790,강원도 겨울 축제 홍천 평창 화천 가볼만한곳 얼음 낚시 체험 가볼만한곳,강원,겨울 축제 얼음 낚시 체험,1,4
...,...,...,...,...,...,...
49651,65ee9fa7c27eda916fc759ae,정방폭포 제주도 가볼만한 곳 제주여행,제주,정방폭포,5,0
49652,65ee9fa7c27eda916fc759af,[제주맛집] 먹방! 제주쇠소깍 오메기식당,제주,맛집 먹방 식당,4,0
49654,65ee9fa8c27eda916fc759b1,제주시 오라동 유채꽃밭 (오라동 메밀꽃밭),제주,유채꽃 메밀 꽃밭,5,1
49657,65ee9fa9c27eda916fc759b4,집에서 만나는 전국 맛집,제주,전국 맛집,4,0
